![](http://sigdelta.com/assets/images/sages-sd-logo.png)

# Analiza danych tekstowych w Python


## Gensim i word embeddings

W tym notebooku przedstawiamy pakiet Gensim w kontekście word embeddings

## Trenowanie własnych w2v

Wykorzystamy ten sam korpus co przy nauce tagerów w poprzednim notatniku.

In [ ]:
import nltk
corpus = nltk.corpus.brown
sentences = corpus.sents()

Zdania bez dużych liter:

In [ ]:
sentences_lower = [[w.lower() for w in s] for s in sentences]

W2V można trenować korzystając z modelu [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)

In [ ]:
from gensim.models import Word2Vec

w2v_brown = Word2Vec(sentences, size=100, min_count=5)

In [ ]:
w2v_brown_lower = Word2Vec(sentences_lower, size=100, min_count=5)

Pobranie wektora cech danego słowa jest bardzo proste:

In [ ]:
w2v_brown["system"]

### Zadanie

1. Wytrenuj modele w2v wielokości 50 i 200. Wytrenowane modele zachowaj w osobnych zmiennych.

## Wczytywanie gotowych modeli

Gotowe modele można wczytać z pliku korzystając z klasy Word2Vec lub [KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html)

Wczytamy gotowy, stosunkowo niewielki model [GloVe](https://nlp.stanford.edu/projects/glove/). Samodzielnie pobrany model należy wstępnie przekonwertować narzędziem [gensim.scripts.glove2word2vec](https://radimrehurek.com/gensim/scripts/glove2word2vec.html)

In [ ]:
from gensim.models.keyedvectors import KeyedVectors

# Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 100d)
w2v_path = "w2v/glove.6B.100d.w2vformat.txt"

w2v_glove = KeyedVectors.load_word2vec_format(w2v_path, binary=False)

## Eksploracja modelu w2v

Najbardziej podobne słowa

In [ ]:
w2v_glove.most_similar(positive=['girl'])

In [ ]:
w2v_brown.most_similar(positive=['girl'])

In [ ]:
w2v_brown_lower.most_similar(positive=['girl'])

Miara podobieństwa dwóch słów

In [ ]:
w2v_glove.similarity('dog', 'cat')

Niepasujące słowa

In [ ]:
w2v_glove.doesnt_match("dog cat fish home bird".split())

## Własny tagger korzystający z W2V

Podobnie jak w poprzednim notatniku, dzielimy korpus na zbiór treninowy i testowy. Definiujemy pomocnicze funkcje do transformacji danych.

In [ ]:
brown_tagged_sents = corpus.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]

def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y

Funkcja generująca cechy zwraca tylko jędną cechę: dane słowo

In [ ]:

def features(sentence, index):
    """ sentence: [w1, w2, ...], index: the index of the word """
    return {
        'word': sentence[index].lower(),
#         'prev_word': '' if index == 0 else sentence[index - 1].lower(),
#         'next_word': '' if index == len(sentence) - 1 else sentence[index + 1].lower()
    }

X, y = transform_to_dataset(train_sents)
X_test, y_test = transform_to_dataset(test_sents)
X[0]

Stworzymy niestandardowy `vectorizer` rozszerzający [W2VTransformer](https://radimrehurek.com/gensim/sklearn_api/w2vmodel.html). Umożliwia on konkatenację wektorów w2v poszczególnych cech.

In [ ]:
import numpy as np
import six
from gensim.sklearn_api.w2vmodel import W2VTransformer

class CustomTransformer(W2VTransformer):
    def __init__(self, gensim_model):
        self.gensim_model = gensim_model
        self.size = gensim_model.vector_size
        
    def fit(self, X, y=None):
        return self
        
    def transform(self, words):
        
        X = [[] for _ in range(0, len(words))]
        feat_num = len(words[0])
        for k, v in enumerate(words):
            vectors = []
            for f_key, f_val in v.items():
                vectors.append(self._transform_word(f_val)) 
            X[k] = np.concatenate(vectors)

        return np.reshape(np.array(X), (len(words), self.size*feat_num))

    def _transform_word(self, word):
        try:
            return self.gensim_model[word]
        except KeyError:
            return np.zeros(self.size)
        

Pora na naukę klasyfikatora

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
 
clf = Pipeline([
    ('vectorizer', CustomTransformer(w2v_glove)),
#     ('classifier', DecisionTreeClassifier(criterion='entropy'))
    ('classifier', svm.LinearSVC())
])

limit = 10000 # Use only the first 10K samples 
 
clf.fit(X[:limit], y[:limit])   
print("Classifier ready!")

In [ ]:
print("Accuracy:", clf.score(X_test, y_test))

### Zadanie

1. Porównaj skuteczność klasyfikacji z wykorzystaniem samodzielnie nauczonego i gotowego modelu w2v.
2. Sprawdź wpływ wielkości wektora samodzielnie nauczonych modeli w2v.
3. Sprawdź czy wykorzystanie poprzedniego i następnego słowa do generowania cech poprawia skuteczność klasyfikacji. 